
"""
CHECK

- The Missing Value Treatement for the rtgp (runtime in group) variables - espcially for races where NONE of the dogs have a runtime

More Variables :

Total Races from current race
- Races in the last 7/14/28/60/90/180/365/730 days 
- Races won in the 7/14/28/60/90/180/365/730 days
- Races placed in the 7/14/28/60/90/180/365/730 days


"""


# Import the code 


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/Dataprep'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/Code', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [20]:
DEBUG = False

In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:

# List files in the directory
files = os.listdir(fileloc)

# Print the file names
for file in files:
    print(file)
    

dataprep_categorical_2022-11-25.csv
dataprep_categorical_2022-11-27.csv
dataprep_categorical_2022-11-28.csv
dataprep_categorical_2022-12-03.csv
dataprep_categorical_2023-02-22.csv
dataprep_performance_2022-11-27.csv
dataprep_performance_2022-11-28.csv
dataprep_performance_2022-12-03.csv
dataprep_performance_2023-02-22.csv
dataprep_raceprofile_2022-10-29.csv
dataprep_raceprofile_2022-11-27.csv
dataprep_raceprofile_2022-11-28.csv
dataprep_raceprofile_2022-12-03.csv
dataprep_raceprofile_2023-02-22.csv
dataprep_speed_profile_2022-10-27.csv
dataprep_speed_profile_2022-11-27.csv
dataprep_speed_profile_2022-11-28.csv
dataprep_speed_profile_2022-12-03.csv
dataprep_speed_profile_2023-02-22.csv
dataprep_TRNR_performance_2022-12-10.csv
dataprep_TRNR_performance_2023-02-22.csv
dataset_base_2022-10-27.csv
dataset_base_2022-11-25.csv
dataset_base_2022-11-27.csv
dataset_base_2022-11-28.csv
dataset_base_2022-12-03.csv
dataset_base_2022-12-12.csv
dataset_base_2022-12-16.csv
dataset_base_2023-02-09.csv


## IMPORTING THE RAW BASE

In [8]:

df.ft_raw = pd.read_csv(f'{fileloc}/raw_base_2023-06-05.csv', parse_dates = True)

print(df.ft_raw.shape)
print(df.ft_raw.columns.values.tolist())



(725080, 33)
['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos']


In [12]:

df_base_sp = df.ft_raw.copy()


### B) Create Variable for Speed Profile

In [13]:

# Sort for Easy QC
df_base_sp.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)


In [14]:

df_base_sp[df_base_sp['@id_DOG'].isin([108390981])]


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,y_runtime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position,y_margin,y_pos
0,108390981,4,DOLLY DOES IT,5.0,5,30.4,$11.00,NaN,6.00,0.14,0,0,B3,NaN,26.43,80.0,626271182,92892,M Elphinstone,626271182,10,GREAT NORTHERN,03:14PM,452m,Mixed 4/5,Devonport,05 Jan 21,2021-01-05,11.0,8,4,6.00,4
1,108390981,5,DOLLY DOES IT,4.0,4,30.2,$7.00,NaN,6.00,1.14,0,0,NaN,NaN,26.40,0.0,629797242,92892,M Elphinstone,629797242,10,GREAT NORTHERN,03:09PM,452m,Grade 5,Devonport,12 Jan 21,2021-01-12,7.0,7,5,6.00,5
2,108390981,1,DOLLY DOES IT,5.0,5,30.2,$7.00,NaN,5.25,NaN,0,0,NaN,8.1,26.20,1890.0,629797513,92892,M Elphinstone,629797513,4,SUREPICK,01:27PM,452m,Grade 5,Devonport,19 Jan 21,2021-01-19,7.0,7,1,5.25,1
3,108390981,7,DOLLY DOES IT,1.0,1,30.0,$7.00,NaN,10.50,2.00,0,0,NaN,NaN,26.48,0.0,638810808,92892,M Elphinstone,638810808,8,LADBROKES,02:22PM,452m,Mixed 4/5 Heat,Devonport,16 Feb 21,2021-02-16,7.0,8,7,10.50,7
4,108390981,8,DOLLY DOES IT,8.0,8,29.9,$91.00,NaN,18.50,3.57,0,0,SO,NaN,26.91,0.0,643554706,92892,M Elphinstone,643554706,8,NEW RECRUIT 2012,02:22PM,452m,Grade 4,Devonport,02 Mar 21,2021-03-02,91.0,8,8,18.50,8


In [15]:

# ADDITIONAL VARIABLES
doglevel = ['@id_DOG']

# Distance and Speed in each race
#df_base_sp['x_dist'] = df_base_sp['Distance'].apply(lambda x : x.replace('m','')).astype(int)
df_base_sp['x_dist'] = df_base_sp['Distance'].str.replace('m','').astype(int)

df_base_sp['speed'] = (df_base_sp['x_dist'] / df_base_sp['y_runtime'])*1000

# Flag for getting a total race count
df_base_sp['f_1'] = 1

# Total Races
df_base_sp['x_races_tot'] = df_base_sp.groupby(doglevel).cumcount()


In [16]:


"""
In the given code, you're computing rolling statistics (mean, min, max, and standard deviation) for the previous speed of each dog. 
    To optimize the calculation of these rolling statistics, 
    you could compute all of them in a single operation by using the .agg() function of a rolling object. 
    This way, you perform the grouping, shifting, and rolling operations just once, 
    and then calculate all the statistics you need.

Also, keep in mind that groupby operations can be computationally expensive. 
    The key is to minimize the number of times you perform a groupby operation on your dataframe.
"""


# Average Speed until prev race
df_base_sp['speed_prev'] = df_base_sp.groupby(doglevel)['speed'].shift(1)


#df_base_sp['speed_avg'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).mean().reset_index(0,drop=True)
#df_base_sp['speed_min'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).min().reset_index(0,drop=True)
#df_base_sp['speed_max'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)
#df_base_sp['speed_std'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)

rolling = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1)

df_base_sp[['speed_avg', 'speed_min', 'speed_max', 'speed_std']] = rolling.agg(['mean', 'min', 'max', 'std']).reset_index(0, drop=True)


df_base_sp['speed_range'] = df_base_sp['speed_max'] - df_base_sp['speed_min']


print(datetime.now())
      

2023-06-06 08:35:28.344097


In [19]:

# ADDITIONAL VARIABLES
grouplevel = ['@id_DOG','x_dist_gp']

# GROUPED X_DIST 
#df_base_sp['x_dist_gp'] = df_base_sp['x_dist'].apply(lambda x : 300 if x < 330 else 350 if x < 380 else 400 if x < 420 else 450 if x < 485 else 500 if x < 550 else 600 if x < 650 else 700 if x < 750 else -999)
df_base_sp['x_dist_gp'] = pd.cut(df_base_sp['x_dist'], bins=[0, 330, 380, 420, 485, 550, 650, 750, np.inf], labels=[300, 350, 400, 450, 500, 600, 700, -999], right=False)
df_base_sp['x_dist_gp'] = df_base_sp['x_dist_gp'].astype(int)

df_base_sp['x_runtm_gp'] = ( df_base_sp['y_runtime'] / df_base_sp['x_dist'] ) * df_base_sp['x_dist_gp']

df_base_sp['x_rtgp_prev'] = df_base_sp.groupby(grouplevel)['x_runtm_gp'].shift(1)


# #df_base_sp['x_rtgp_avg'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().reset_index(drop=True)
# df_base_sp['x_rtgp_avg_v0'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().values

# #df_base_sp['x_rtgp_min'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().reset_index(drop=True)
# df_base_sp['x_rtgp_min_v0'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().values

# #df_base_sp['x_rtgp_max'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)
# df_base_sp['x_rtgp_max_v0'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().values

# df_base_sp['x_rtgp_range_v0'] = df_base_sp['x_rtgp_max_v0'] - df_base_sp['x_rtgp_min_v0']

# #df_base_sp['x_rtgp_std'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)
# df_base_sp['x_rtgp_std_v0'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().values


rolling = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods=1)

df_base_sp[['x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_std']] = rolling.agg(['mean', 'min', 'max', 'std']).values

df_base_sp['x_rtgp_range'] = df_base_sp['x_rtgp_max'] - df_base_sp['x_rtgp_min']

print(datetime.now())


2023-06-06 08:40:11.935572


In [24]:

#DEBUG = True

#QC old and new code 
if DEBUG:
    print(df_base_sp['x_rtgp_avg_v0'].equals(df_base_sp['x_rtgp_avg']))
    print(df_base_sp['x_rtgp_min_v0'].equals(df_base_sp['x_rtgp_min']))
    print(df_base_sp['x_rtgp_max_v0'].equals(df_base_sp['x_rtgp_max']))
    print(df_base_sp['x_rtgp_range_v0'].equals(df_base_sp['x_rtgp_range']))
    print(df_base_sp['x_rtgp_std_v0'].equals(df_base_sp['x_rtgp_std']))



True
True
True
True
True


In [25]:

#df_base_sp.head(25)

#df_base_sp[ df_base_sp['@id_DOG'].isin([108390981,108391383,108391431]) ].head(25)

df_base_sp[ df_base_sp['x_runtm_gp'].isna()].head()



,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,y_runtime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,...,y_margin,y_pos,x_dist,speed,f_1,x_races_tot,speed_prev,speed_avg,speed_min,speed_max,speed_std,speed_range,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg_v0,x_rtgp_min_v0,x_rtgp_max_v0,x_rtgp_range_v0,x_rtgp_std_v0,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_std,x_rtgp_range


In [26]:

print(df_base_sp.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_std', 'speed_range', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg_v0', 'x_rtgp_min_v0', 'x_rtgp_max_v0', 'x_rtgp_range_v0', 'x_rtgp_std_v0', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_std', 'x_rtgp_range']


In [27]:

#temp = df_base_sp.groupby('x_dist').agg( {'@id_RACE':'nunique'} ).reset_index()
#temp.to_csv(f'{fileloc}/race_distance.csv', index = False)
#temp.sort_values(by = 'index').head()


In [28]:

print( len(df_base_sp['@id_DOG'].unique().tolist()) )

print(df_base_sp['speed'].isna().sum())
print(df_base_sp['speed_prev'].isna().sum())


30513
0
30513


In [30]:

print(pd.DataFrame(df_base_sp.dtypes))


                    0
@id_DOG         int64
Place          object
DogName        object
Box           float64
Rug             int64
...               ...
x_rtgp_avg    float64
x_rtgp_min    float64
x_rtgp_max    float64
x_rtgp_std    float64
x_rtgp_range  float64

[56 rows x 1 columns]


In [32]:

print(pd.DataFrame(df_base_sp.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  y_runtime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    725080        0   100612  97952       0    351207       119772          0      226219       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  ...  y_margin  y_pos  x_dist  speed  f_1  x_races_tot  speed_prev  speed_avg  speed_min  speed_max  speed_std  speed_range  x_dist_gp  x_runtm_gp  \
0        15         0         0          0  ...         0      0       0      0    0            0       30513      30513      30513      30513      59508        30513          0           0   

   x_rtgp_prev  x_rtgp_avg_v0  x_rtgp_min_v0  x_rtgp_max_v0  x_rtgp_range_v0  x_rtgp_std_v0  x_rtgp_avg  x_rtgp_min  x_rtgp_max  x_rtgp_std  x_rtgp_range  
0        89140          89140          89140          89


### C) Treat Missing Values


In [33]:

# All Columns Realted to Average Speed

#df_base_sp['x_no_prev_races'] = df_base_sp['speed_prev'].apply( lambda x : 1 if pd.isna(x) else 0)
'''

The condition pd.isna(x) can be applied directly to the entire pandas series df_base_sp['speed_prev'],
    without the need for apply() and lambda functions.
    The isna() function returns a boolean series, which is converted to integer using astype(int).
'''

df_base_sp['x_no_prev_races'] = df_base_sp['speed_prev'].isna().astype(int)
print( df_base_sp['x_no_prev_races'].value_counts() )


# df_base_sp['x_speed_prev'] = df_base_sp['speed_prev'].fillna(0)
# df_base_sp['x_speed_avg'] = df_base_sp['speed_avg'].fillna(0)
# df_base_sp['x_speed_min'] = df_base_sp['speed_min'].fillna(0)
# df_base_sp['x_speed_max'] = df_base_sp['speed_max'].fillna(0)

df_base_sp[['x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_std']] = df_base_sp[['speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_std']].fillna(0)
df_base_sp['x_speed_range'] = df_base_sp['speed_range'].fillna(-1)


#df_base_sp['x_speed_std'] = df_base_sp.apply(lambda x : -1 if x.x_no_prev_races == 1 else x.x_speed_std, axis = 1  )
"""
Instead of using apply() to conditionally assign a value to the 'x_speed_std' column, 
    I've used boolean indexing (df_base_sp.loc[mask, 'x_speed_std'] = -1). 
    This method is typically faster because it uses pandas' vectorized operations.
"""
mask = df_base_sp['x_no_prev_races'] == 1
df_base_sp.loc[mask, 'x_speed_std'] = -1

print(datetime.now())


0    694567
1     30513
Name: x_no_prev_races, dtype: int64
2023-06-06 08:49:16.955697


In [34]:

# All Columns Related to RunTime in Group

#df_base_sp['x_no_prev_r_in_dist'] = df_base_sp['x_rtgp_prev'].apply( lambda x : 1 if pd.isna(x) else 0)
df_base_sp['x_no_prev_r_in_dist'] = df_base_sp['x_rtgp_prev'].isna().astype(int)

print( df_base_sp['x_no_prev_r_in_dist'].value_counts() )

print(datetime.now())


0    635940
1     89140
Name: x_no_prev_r_in_dist, dtype: int64
2023-06-06 08:50:37.842575


In [44]:


# Using missing value treatment at the RACE level.


# rtgp_mvt = df_base_sp.groupby('@id_RACE').agg({ 'x_rtgp_prev':'mean', 'x_rtgp_avg':'mean', 'x_rtgp_min':'mean', 'x_rtgp_max':'mean', 'x_rtgp_range':'mean', 'x_rtgp_std':'mean'})\
#             .reset_index()\
#             .rename(columns = {'x_rtgp_prev':'mvt_rtgp_prev', 'x_rtgp_avg':'mvt_rtgp_avg', 'x_rtgp_min':'mvt_rtgp_min', 'x_rtgp_max':'mvt_rtgp_max', 'x_rtgp_range':'mvt_rtgp_range', 'x_rtgp_std':'mvt_rtgp_std'})

agg_columns = ['x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std']
rename_dict = {old: 'mvt_' + old for old in agg_columns}
print(rename_dict)
print()

rtgp_mvt = (df_base_sp.groupby('@id_RACE')[agg_columns]
            .mean()
            .reset_index()
            .rename(columns=rename_dict))


print(pd.DataFrame(rtgp_mvt.isna().sum()).transpose())
print()
print(rtgp_mvt.shape)


{'x_rtgp_prev': 'mvt_x_rtgp_prev', 'x_rtgp_avg': 'mvt_x_rtgp_avg', 'x_rtgp_min': 'mvt_x_rtgp_min', 'x_rtgp_max': 'mvt_x_rtgp_max', 'x_rtgp_range': 'mvt_x_rtgp_range', 'x_rtgp_std': 'mvt_x_rtgp_std'}

   @id_RACE  mvt_x_rtgp_prev  mvt_x_rtgp_avg  mvt_x_rtgp_min  mvt_x_rtgp_max  mvt_x_rtgp_range  mvt_x_rtgp_std
0         0              865             599             599             599               599            1687

(100422, 7)


In [45]:

print(df_base_sp.columns.values.tolist())
print()
print(df_base_sp.shape)


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_std', 'speed_range', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg_v0', 'x_rtgp_min_v0', 'x_rtgp_max_v0', 'x_rtgp_range_v0', 'x_rtgp_std_v0', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_std', 'x_rtgp_range', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_std', 'x_speed_range', 'x_no_prev_r_in_dist']

(725080, 64)


In [46]:

print(df_base_sp.isna().sum().to_frame().T, "\n\n\n")

print(df_base_sp.shape)
df.ft_mvt = df_base_sp.merge(rtgp_mvt, on = '@id_RACE', how = 'left')
print(df.ft_mvt.shape)

# df.ft_mvt['x_rtgp_prev'].fillna(df.ft_mvt['mvt_rtgp_prev'], inplace=True)

# df.ft_mvt['x_rtgp_avg'].fillna(df.ft_mvt['mvt_rtgp_avg'], inplace=True)
# df.ft_mvt['x_rtgp_min'].fillna(df.ft_mvt['mvt_rtgp_min'], inplace=True)
# df.ft_mvt['x_rtgp_max'].fillna(df.ft_mvt['mvt_rtgp_max'], inplace=True)

# df.ft_mvt['x_rtgp_range'].fillna(df.ft_mvt['mvt_rtgp_range'], inplace=True)
# df.ft_mvt['x_rtgp_std'].fillna(df.ft_mvt['mvt_rtgp_std'], inplace=True)

fillna_cols = ['x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std']
for col in fillna_cols:
    df.ft_mvt[col].fillna(df.ft_mvt['mvt_' + col], inplace=True)

print("\n\n\n")
print(df.ft_mvt.isna().sum().to_frame().T)


# MIGHT NEED TO CHECK THIS AGAIN

# df.ft_mvt['x_rtgp_prev'].fillna(0, inplace=True)

# df.ft_mvt['x_rtgp_avg'].fillna(0, inplace=True)
# df.ft_mvt['x_rtgp_min'].fillna(0, inplace=True)
# df.ft_mvt['x_rtgp_max'].fillna(0, inplace=True)

# df.ft_mvt['x_rtgp_range'].fillna(0, inplace=True)
# df.ft_mvt['x_rtgp_std'].fillna(0, inplace=True)

df.ft_mvt[fillna_cols] = df.ft_mvt[fillna_cols].fillna(0)

print(pd.DataFrame(df.ft_mvt.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  y_runtime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    725080        0   100612  97952       0    351207       119772          0      226219       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  ...  speed_min  speed_max  speed_std  speed_range  x_dist_gp  x_runtm_gp  x_rtgp_prev  x_rtgp_avg_v0  x_rtgp_min_v0  x_rtgp_max_v0  x_rtgp_range_v0  \
0        15         0         0          0  ...      30513      30513      59508        30513          0           0        89140          89140          89140          89140            89140   

   x_rtgp_std_v0  x_rtgp_avg  x_rtgp_min  x_rtgp_max  x_rtgp_std  x_rtgp_range  x_no_prev_races  x_speed_prev  x_speed_avg  x_speed_min  x_speed_max  x_speed_std  x_speed_range  x_no_prev_r_in_dist  
0       

In [47]:

#df.ft_mvt[ df.ft_mvt.mvt_rtgp_prev.isna() | df.ft_mvt.x_rtgp_prev.isna()  ].head(25)

df.ft_mvt[ df.ft_mvt['@id_RACE'].isin([626271182, 626218628, 626240593])].sort_values(by = ['@id_RACE','Place']).head(25)


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,y_runtime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,...,x_rtgp_prev,x_rtgp_avg_v0,x_rtgp_min_v0,x_rtgp_max_v0,x_rtgp_range_v0,x_rtgp_std_v0,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_std,x_rtgp_range,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_std,x_speed_range,x_no_prev_r_in_dist,mvt_x_rtgp_prev,mvt_x_rtgp_avg,mvt_x_rtgp_min,mvt_x_rtgp_max,mvt_x_rtgp_range,mvt_x_rtgp_std
16820,251076212,1,BURN OUT BOY,5.0,5,33.2,$2.60F,NaN,1.00,NaN,211,0,M21c1 Sw,6.62,22.23,1100.0,626218628,68894,G Price,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
184030,415996356,2,SCARY FUN,3.0,3,27.2,$4.00,NaN,1.00,1.14,332,0,Mc3c3c2,6.77,22.31,330.0,626218628,112106,L Isaac,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
31810,295446742,3,METRE EATER,2.0,2,33.1,$9.90,NaN,4.25,3.00,744,0,Sc7c44,6.88,22.52,165.0,626218628,72510,B McPherson,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
7203,219044234,4,REPEAT SET,8.0,8,30.4,$12.00,NaN,5.50,1.43,656,0,M65w6,6.84,22.62,55.0,626218628,127397,C Bainbridge,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
14794,246692403,5,KABARANGI,1.0,1,24.1,$40.60,NaN,5.75,0.29,565,0,Q5c65,6.84,22.64,0.0,626218628,291086,T Mitchem,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
5,108391383,6,SIMPLY GIFTED,6.0,6,24.8,$46.50,NaN,7.25,1.43,877,0,Sc8c77,6.93,22.74,0.0,626218628,291086,T Mitchem,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
10885,231587263,7,ABBEY SIOBHAN,4.0,4,30.8,$5.60,NaN,9.00,1.71,123,0,Q12cc3,6.59,22.86,0.0,626218628,102996,M Stewart,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
28292,289026900,8,CROWN BENJI,7.0,9,32.4,$6.80,NaN,18.25,9.29,488,0,M48to8 I-21,6.82,23.51,0.0,626218628,83951,R Noden,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
227605,434800047,1,IVY CONNIVING,8.0,8,28.2,$2.20F,NaN,8.75,NaN,111,0,111,8.20,22.55,1600.0,626240593,108360,S Sultana,626240593,6,STEELINE RIVERSTONE,04:52PM,400m,Mixed 4/5,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
17179,253088672,2,SHIMMY SPARKLES,3.0,3,27.0,$19.00,NaN,8.75,8.71,544,0,544,8.50,23.16,500.0,626240593,114260,L Azzopardi,626240593,6,STEELINE RIVERSTONE,04:52PM,400m,Mixed 4/5,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:

print(df.ft_mvt.describe())


            @id_DOG            Box            Rug         Weight  Handicap        Margin1        Margin2         Checks    SplitMargin      y_runtime      Prizemoney        RaceId      TrainerId  \
count  7.250800e+05  725080.000000  725080.000000  725079.000000       0.0  725080.000000  624468.000000  725080.000000  605308.000000  725080.000000   498861.000000  7.250800e+05  725080.000000   
mean   4.866356e+08       4.477743       4.584582      29.935174       NaN       6.483733       2.071076       0.426676       7.265410      24.363969      441.621810  7.640069e+08  111702.449377   
std    1.219313e+08       2.343451       2.431397       3.312836       NaN       8.260129       7.241565       1.439907       2.526565       4.941794     2912.322446  7.786416e+07   78747.029834   
min    1.083910e+08       1.000000       1.000000      18.700000       NaN       0.000000       0.000000       0.000000       0.000000      10.000000        0.000000  6.238619e+08    6418.000000   
25%    4.1

In [49]:

print( df.ft_mvt.columns.values.tolist() )


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_std', 'speed_range', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg_v0', 'x_rtgp_min_v0', 'x_rtgp_max_v0', 'x_rtgp_range_v0', 'x_rtgp_std_v0', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_std', 'x_rtgp_range', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_std', 'x_speed_range', 'x_no_prev_r_in_dist', 'mvt_x_rtgp_prev', 'mvt_x_rtgp_avg', 'mvt_x_rtgp_min', 'mvt_x_rtgp_max', 'mvt_x_rtgp_range', 'mvt_x_rtgp_std']


In [50]:

print(datetime.now().date())


2023-06-06


In [52]:

# 'TrainerName', 'Track' have been dummified
cols_for_drop = ['Box','Rug','Weight'\
                 #,'Handicap'
                , 'Margin1','Margin2','PIR','Checks','Comments','SplitMargin','Prizemoney','RaceId','TrainerId','RaceTime','Distance','RaceGrade','date', 'f_1','speed'\
                , 'Place', 'StartPrice'\
                , 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std'\
                , 'DogName', 'TrainerName', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'position','y_runtime'\
                , 'x_dist','ft_sp', 'x_dist', 'x_num_dogs'\
                , 'mvt_x_rtgp_prev', 'mvt_x_rtgp_avg', 'mvt_x_rtgp_min' ,'mvt_x_rtgp_max' ,'mvt_x_rtgp_range', 'mvt_x_rtgp_std'\
                , 'y_margin','y_pos']

## SHOULD BE THE LAST STEP OR a Separate Code
#df.ft1 = df.ft.copy()
exp_ds = df.ft_mvt.drop(columns = cols_for_drop)

# removed the Categorical columns from the original datast, adds the dummy variables, creates a new dataset
print(exp_ds.columns)


Index(['@id_DOG', 'Handicap', '@id_RACE', 'x_races_tot', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg_v0', 'x_rtgp_min_v0', 'x_rtgp_max_v0', 'x_rtgp_range_v0', 'x_rtgp_std_v0', 'x_rtgp_avg',
       'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_std', 'x_rtgp_range', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_std', 'x_speed_range',
       'x_no_prev_r_in_dist'],
      dtype='object')


In [53]:

exp_ds.head()


,@id_DOG,Handicap,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg_v0,x_rtgp_min_v0,x_rtgp_max_v0,x_rtgp_range_v0,x_rtgp_std_v0,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_std,x_rtgp_range,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_std,x_speed_range,x_no_prev_r_in_dist
0,108390981,NaN,626271182,0,450,26.313053,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1
1,108390981,NaN,629797242,1,450,26.283186,26.313053,26.313053,26.313053,26.313053,0.000000,NaN,26.313053,26.313053,26.313053,0.000000,0.000000,0,17101.778282,17101.778282,17101.778282,17101.778282,0.000000,0.000000,0
2,108390981,NaN,629797513,2,450,26.084071,26.283186,26.298119,26.283186,26.313053,0.029867,0.021119,26.298119,26.283186,26.313053,0.021119,0.029867,0,17121.212121,17111.495202,17101.778282,17121.212121,13.741799,19.433839,0
3,108390981,NaN,638810808,3,450,26.362832,26.084071,26.226770,26.084071,26.313053,0.228982,0.124480,26.226770,26.084071,26.313053,0.124480,0.228982,0,17251.908397,17158.299600,17101.778282,17251.908397,81.647864,150.130115,0
4,108390981,NaN,643554706,4,450,26.790929,26.362832,26.260785,26.084071,26.362832,0.278761,0.122305,26.260785,26.084071,26.362832,0.122305,0.278761,0,17069.486405,17136.096301,17069.486405,17251.908397,80.101155,182.421992,0


### EXPORTING THE SPEED PROFILE csv

In [54]:

#keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Track','RaceGrade','Distance','TrainerName','RunTime','Place','x_races_tot']
#'Event_Dt', 

cols_for_drop = []

todaydt = datetime.now().date()
exp_fname = f'dataprep_speed_profile_{todaydt}.csv'


In [55]:

print(pd.DataFrame(exp_ds.isna().sum()).transpose())


   @id_DOG  Handicap  @id_RACE  x_races_tot  x_dist_gp  x_runtm_gp  x_rtgp_prev  x_rtgp_avg_v0  x_rtgp_min_v0  x_rtgp_max_v0  x_rtgp_range_v0  x_rtgp_std_v0  x_rtgp_avg  x_rtgp_min  x_rtgp_max  \
0        0    725080         0            0          0           0            0          89140          89140          89140            89140         160923           0           0           0   

   x_rtgp_std  x_rtgp_range  x_no_prev_races  x_speed_prev  x_speed_avg  x_speed_min  x_speed_max  x_speed_std  x_speed_range  x_no_prev_r_in_dist  
0           0             0                0             0            0            0            0            0              0                    0  


In [56]:
print(exp_ds.shape)

(725080, 25)


In [57]:

print(exp_ds.columns.values.tolist())


['@id_DOG', 'Handicap', '@id_RACE', 'x_races_tot', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg_v0', 'x_rtgp_min_v0', 'x_rtgp_max_v0', 'x_rtgp_range_v0', 'x_rtgp_std_v0', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_std', 'x_rtgp_range', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_std', 'x_speed_range', 'x_no_prev_r_in_dist']


In [58]:

exp_ds.head()


,@id_DOG,Handicap,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg_v0,x_rtgp_min_v0,x_rtgp_max_v0,x_rtgp_range_v0,x_rtgp_std_v0,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_std,x_rtgp_range,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_std,x_speed_range,x_no_prev_r_in_dist
0,108390981,NaN,626271182,0,450,26.313053,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1
1,108390981,NaN,629797242,1,450,26.283186,26.313053,26.313053,26.313053,26.313053,0.000000,NaN,26.313053,26.313053,26.313053,0.000000,0.000000,0,17101.778282,17101.778282,17101.778282,17101.778282,0.000000,0.000000,0
2,108390981,NaN,629797513,2,450,26.084071,26.283186,26.298119,26.283186,26.313053,0.029867,0.021119,26.298119,26.283186,26.313053,0.021119,0.029867,0,17121.212121,17111.495202,17101.778282,17121.212121,13.741799,19.433839,0
3,108390981,NaN,638810808,3,450,26.362832,26.084071,26.226770,26.084071,26.313053,0.228982,0.124480,26.226770,26.084071,26.313053,0.124480,0.228982,0,17251.908397,17158.299600,17101.778282,17251.908397,81.647864,150.130115,0
4,108390981,NaN,643554706,4,450,26.790929,26.362832,26.260785,26.084071,26.362832,0.278761,0.122305,26.260785,26.084071,26.362832,0.122305,0.278761,0,17069.486405,17136.096301,17069.486405,17251.908397,80.101155,182.421992,0


In [59]:

exp_ds.describe()


,@id_DOG,Handicap,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg_v0,x_rtgp_min_v0,x_rtgp_max_v0,x_rtgp_range_v0,x_rtgp_std_v0,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_std,x_rtgp_range,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_std,x_speed_range,x_no_prev_r_in_dist
count,7.250800e+05,0.0,7.250800e+05,725080.000000,725080.000000,725080.000000,725080.000000,635940.000000,635940.000000,635940.000000,635940.000000,564157.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000,725080.000000
mean,4.866356e+08,NaN,7.640069e+08,20.092517,414.921327,24.288610,24.075946,24.302320,23.937817,24.798601,0.860784,0.323198,24.031286,23.678688,24.508748,0.314599,0.830060,0.042082,16383.901335,16379.381781,15866.486933,16795.481739,261.329562,928.952724,0.122938
std,1.219313e+08,NaN,7.786416e+07,17.819576,78.150315,4.646626,5.097870,4.542681,4.518502,4.598102,0.742289,0.204947,4.809930,4.772877,4.880562,0.194952,0.715255,0.200777,3459.546514,3447.694940,3362.879397,3559.951694,188.917709,687.335695,0.328366
min,1.083910e+08,NaN,6.238619e+08,0.000000,-999.000000,-62.082592,0.000000,9.950249,9.950249,9.950249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000
25%,4.159944e+08,NaN,6.985139e+08,6.000000,350.000000,20.713636,20.626027,20.732353,20.459091,21.243202,0.415891,0.219818,20.651465,20.367123,21.112573,0.220017,0.396752,0.000000,16806.722689,16883.241729,16302.016302,17248.722317,191.098862,585.763208,0.000000
50%,4.791464e+08,NaN,7.654781e+08,15.000000,400.000000,23.660000,23.630769,23.611670,23.250633,24.179551,0.810350,0.295785,23.567677,23.210000,24.105674,0.293483,0.778947,0.000000,17089.843750,17105.326015,16596.072391,17526.289434,251.681596,897.711790,0.000000
75%,5.620734e+08,NaN,8.292900e+08,29.000000,450.000000,27.114849,27.010441,27.088359,26.740000,27.751740,1.180000,0.387710,26.711137,26.380000,27.261021,0.377961,1.141768,0.000000,17353.579176,17289.665913,16833.926837,17760.617761,319.896873,1197.750585,0.000000
max,2.145360e+09,NaN,8.978632e+08,147.000000,700.000000,384.582524,61.747368,46.248951,46.248951,61.747368,39.167368,10.132302,46.248951,46.248951,61.747368,10.132302,39.167368,1.000000,40200.000000,40200.000000,40200.000000,40200.000000,16614.431948,23826.557771,1.000000


In [62]:
print(datetime.now())

2023-06-06 09:03:38.761578


In [60]:

exp_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)


In [63]:

print(datetime.now())
print('THE END')


2023-06-06 09:03:48.861259
THE END
